In [1]:
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

"""
循环神经网络 RNN
在这里的实现是  一个时间步一个时间步  来训练的
"""

# 迭代器每次给出 batch_size 组数据
# 每一组中单个序列的长度为 num_steps 
batch_size, num_steps = 32, 35

# 每次从 train_iter 中取出一个元素
# 元组包含两个元素一个是 X 一个是 Y
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [2]:
for X, Y in train_iter:
    print('Matrix X is ', X.shape)
    print(X)
    print('Matrix Y is ', Y.shape)
    print(Y)
    # y 是 Y.T 拉直后的样子，顺序拼接
    # 相对与 Y 来说就是把 Y 的每一列依次拼接成 1 维的向量
    y = Y.T.reshape(-1)
    print(y, ' and y.long() is ', y.long())
    print('------------------')

Matrix X is  torch.Size([32, 35])
tensor([[ 1,  3,  5,  ...,  5, 13,  2],
        [18,  9,  3,  ..., 20,  4,  8],
        [ 9,  1,  4,  ..., 11,  1, 12],
        ...,
        [ 1, 15,  7,  ...,  1, 21, 14],
        [10, 19,  8,  ..., 14,  8,  3],
        [13,  2, 15,  ...,  1,  4,  6]])
Matrix Y is  torch.Size([32, 35])
tensor([[ 3,  5, 13,  ..., 13,  2,  1],
        [ 9,  3,  1,  ...,  4,  8,  8],
        [ 1,  4,  1,  ...,  1, 12,  4],
        ...,
        [15,  7,  6,  ..., 21, 14,  3],
        [19,  8,  3,  ...,  8,  3,  1],
        [ 2, 15,  9,  ...,  4,  6, 11]])
tensor([ 3,  9,  1,  ...,  3,  1, 11])  and y.long() is  tensor([ 3,  9,  1,  ...,  3,  1, 11])
------------------
Matrix X is  torch.Size([32, 35])
tensor([[ 1,  3, 10,  ..., 22,  2,  6],
        [ 8,  2, 11,  ...,  5,  6, 18],
        [ 4, 25,  5,  ..., 10,  1,  3],
        ...,
        [ 3, 21,  2,  ...,  4, 15,  2],
        [ 1, 21,  2,  ...,  8,  1, 14],
        [11,  1,  8,  ...,  1,  7,  6]])
Matrix Y is  torch.Si

In [3]:
# one-hot 编码就是 概率分布
# 0 号元素是正确的，所以第 0 号元素的概率为 1 
# 2 号元素是正确的，所以第 2 号元素的概率为 1
F.one_hot(torch.tensor([0, 2]), len(vocab))

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

In [4]:
# 小批量数据的形状是 （批量大小，时间步数）
# 28 代表种类有 28 种,在对应位置上才为 1
# 共有 2 个序列，每个序列的长度为 5
X = torch.arange(10).reshape((2, 5))
F.one_hot(X.T, 28).shape 

torch.Size([5, 2, 28])

In [5]:
# 转置后形状变成了 （时间步数，批量大小）
# 5 行 ==> 5 个时间点
# 2 列 ==> 2 个序列
# 从 ont-hot 角度看
# 它把每行的 2 个元素都展开成了 28 位长的 ont-hot 数组

# (时间步数，批量大小，ont-hot 数组长度)
F.one_hot(X.T, 28)

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]],

        [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0

In [6]:
# 小批量数据的形状是 （批量大小，时间步数）
# 28 代表种类有 28 种,在对应位置上才为 1
X = torch.arange(4).reshape((2, 2))
X

tensor([[0, 1],
        [2, 3]])

In [7]:
X = torch.arange(10).reshape((2, 5))

In [8]:
F.one_hot(X, 10)

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]])

In [9]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        # 初始化学习参数
        return torch.randn(size=shape, device=device) * 0.01

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))
    # print('W_xh.shape is ', W_xh.shape)

    W_hh = normal((num_hiddens, num_hiddens))
    # print('W_hh.shape is ', W_hh.shape)

    b_h = torch.zeros(num_hiddens, device=device)
    # print('b_h.shape is ', b_h.shape)

    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    # print('W_hq.shape is ', W_hq.shape)

    b_q = torch.zeros(num_outputs, device=device)
    # print('b_q.shape is ', b_q.shape)
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

In [10]:
# 初始化隐藏状态
# 虽然第一次的 state 被初始化为了 0 矩阵
# 但是下一次的 state 的计算就不是 0 矩阵了
# 会经过加和的过程计算出来下一次的 state
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [11]:
# W_xh.shape is  torch.Size([28, 512])
# W_hh.shape is  torch.Size([512, 512])
# b_h.shape is  torch.Size([512])
# W_hq.shape is  torch.Size([512, 28])
# b_q.shape is  torch.Size([28])

def rnn(inputs, state, params):
    # inputs 是从数据迭代器中取到的数据 X.T 的 ont-hot 编码后的样子
    # print('inputs.shape is ', inputs.shape)
    # inputs的形状：(时间步数量，批量大小，词表大小)
    # 时间步数是 一个序列的长度
    W_xh, W_hh, b_h, W_hq, b_q = params
    # state 是一个长度为 1 的 tuple
    H, = state
    # print('H.shape is ', H.shape)
    outputs = []
    # X 的形状：(批量大小，词表大小)  词表大小就是共有多少个 词元，不重复的分词后的数量
    # X 是整个批量中所有序列的在某个时间步上的 ont-hot 编码，也即特定于某一个输出的概率分布
    # 这样就特定训练了某一个输出，而且给定了其他的输出应有的值
    # 每一轮循环中计算的是为了计算某一个特定时间步上的预测值
    for X in inputs:
        # 每次拿到的 X 是一个时刻上的所有样本的训练集(批量大小，词表大小) 

        # print('X.shape is ', X.shape)
        # W_xh 的每一列是一个 神经元 ==> 共有 h 个神经元
        # H 的形状是 （批量大小，隐藏层个数）
        # X 的形状是 （批量大小，时间步数）
        # (X, W_xh) （X 的行， W_xh 的列 隐藏层大小）
        # (H, W_hh) （H 的行， W_hh 的列 隐藏层大小）
        # 这里的 X 只是一个时间节点上的取值，计算出来下一次的 H
        # W_hh 存储信息，而 H 用来表示上一次的状态
        # 所以 H 是动态得出的，不需要对 H 进行存储
        # H 是多个 batch_size 个样本的在 X 所表示的 batch_size 个样本的当前上一次的状态取值
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        Y = torch.mm(H, W_hq) + b_q
        # 如果 X 中只含有一个样本
        # 那么 Y 就是一个行向量
        # 如果 X 中有多个样本
        # 那么 Y 是一个矩阵，并且 Y 的每一行是一个样本在某个时间步的概率分布
        outputs.append(Y)
        # print('Matrix Y is ')
        # print(Y)
        # print('Y.shape is ', Y.shape)
        # print('---------------------------------- relate inputs ----------------')
    res = torch.cat(outputs, dim=0)
    # print('res.shape is ', res)
    return res, (H,)

In [12]:
class RNNModelScratch: #@save
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device,
                 get_params, init_state, forward_fn):
                 
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        # 参数 X.shape is (batch_size, num_steps) (批量大小，序列长度:时间步数)
        # vocab 存储了 数值 到 字符串 的对应关系，包含一些特殊字符串
        # 按照英文字母预测则 词表大小 = 26 + 特殊字符
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [13]:
len(vocab)

28

In [14]:
# 检查输出是否符合要求
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                      init_rnn_state, rnn)
state = net.begin_state(X.shape[0], d2l.try_gpu())
print(X)
print(state[0].shape)

# 一个样本就需要一个 state 因此就需要 X.shape[0] 哥 state
# 输入的数据 X 的形状 ==> 批量大小, 时间步数
# print('输入的数据 X 的形状 X.shape is ', X.shape)

# 对上一次的数据完成前向传播后产生一个新的 H
Y, new_state = net(X.to(d2l.try_gpu()), state)

# Y 中每 batch_size 行分别对应了不同的样本在某时间步上的 Y_hat ont-hot 编码
# 一个 batch_size 这么多行是各个样本在同一个时间步上的 y_hat ont-hot 
# new_state 在这一小节是一个长度为 1 的 tuple
Y.shape, len(new_state), new_state[0].shape

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])
torch.Size([2, 512])


(torch.Size([10, 28]), 1, torch.Size([2, 512]))

In [15]:
def predict_ch8(prefix, num_preds, net, vocab, device):  #@save
    """
    在prefix后面生成新字符
    num_preds: 后续要生成多少个 词（本节的次元是 字符）
    """
    # batch_size = 1 ==> 对一个字符串做预测
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]  # 把 prefix[0] 对应的 数值映射 放进 outputs 中
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]:  # 预热期 ==> 把 prefix 中的信息初始化到 state 中
        # y 是从 prefix 的 第1号 元素开始取值的
        # 当 y 取 prefix[1] 的时候，此时 get_input() 得到的输入是 prefix[0] 
        # 刚好是错开的，是没有问题的
        # 之所以 state 也是网络的输入参数
        # 就是使用 state 保存了 prefix 的信息，而 W_hh 是帮助生成 state 用的
        _, state = net(get_input(), state)
        outputs.append(vocab[y])

    # 从这儿开始是预测 prefix 后边的字符
    # 每次都用上一次输出的值作为下一次输入的值
    # 上一次的输出值是一个时间步上的值，而不是一个序列
    # 这刚好和在训练模型时的思路是一致的，使用了时间序列中的
    # 某一个时间步上的值的 one-hot 编码作为输入
    # 因此这里的 RNN 在真正预测的核心部分，它的输入依旧是时间序列中的
    # 某个时间步的值，但用 ont-hot 值来表达
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))

    # 把 index 的值 转化成其 对应的 词（这里是字符）
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [16]:
predict_ch8('time traveller ', 10, net, vocab, d2l.try_gpu())

'time traveller r r r r r '

In [17]:
def grad_clipping(net, theta):  #@save
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [18]:
#@save ch8 = chapter 8 第 8 章
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    """训练网络一个迭代周期（定义见第8章）"""
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)  # 训练损失之和,词元数量
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # state对于nn.GRU是个张量
                state.detach_()
            else:
                # state对于nn.LSTM或对于我们从零开始实现的模型是个张量
                for s in state:
                    s.detach_()
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        # y_hat 是 y 的 one-hot 编码形式
        # y 中的每一个元素展开成为 ont-hot 编码后得到 y_hat
        # y 中的一个元素和 y_hat 的一个元素（ont-hot 向量） 做交叉熵损失
        # pytorch 中的交叉熵损失函数会自动处理 ont-hot 编码计算损失
        l = loss(y_hat, y.long()).mean()

        # y 在这里是一个 1 维的向量
        # y_hat 是一个 2 维的矩阵，并且是 y 的 ont-hot 编码形式

        print('y_hat.shape is ', y_hat.shape, y_hat.dtype)
        print('y.shape is ', y.shape, y.dtype)
        print('y.long().shape is ', y.long().shape, y.long().dtype)
        print('--------------------')

        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调用了mean函数
            updater(batch_size=1)
        metric.add(l * y.numel(), y.numel())
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()

In [19]:
#@save
def train_ch8(net, train_iter, vocab, lr, num_epochs, device,
              use_random_iter=False):
    """训练模型（定义见第8章）"""
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
                            legend=['train'], xlim=[10, num_epochs])
    # 初始化
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)
    # 训练和预测
    for epoch in range(num_epochs):
        ppl, speed = train_epoch_ch8(
            net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0:
            print(predict('time traveller'))
            animator.add(epoch + 1, [ppl])
    print(f'困惑度 {ppl:.1f}, {speed:.1f} 词元/秒 {str(device)}')
    print(predict('time traveller'))
    print(predict('traveller'))

In [20]:
num_epochs, lr = 1, 1
# train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu())

#### 理解 tensor 的 long 方法

In [21]:
X = torch.tensor([1.1, 2.2, 3.3])

In [22]:
X

tensor([1.1000, 2.2000, 3.3000])

In [23]:
X.long()

tensor([1, 2, 3])

#### 理解 torch.cat

In [24]:
X = torch.tensor([
    [1, 2, 3, 4, 5, 6],
    [11, 22, 33, 44, 55, 66]
])

Y = torch.tensor([
    [6, 5, 4, 3, 2, 1],
    [66, 55, 44, 33, 22, 11]
])

torch.cat([X, Y], dim=0)

tensor([[ 1,  2,  3,  4,  5,  6],
        [11, 22, 33, 44, 55, 66],
        [ 6,  5,  4,  3,  2,  1],
        [66, 55, 44, 33, 22, 11]])

#### 理解 loss = nn.CrossEntropyLoss()

In [25]:
loss = nn.CrossEntropyLoss()

# 这里可以为 6
# 因为样本中不一定覆盖了所有的概率分布位置
# 所以有些地方可以不被覆盖
# 比如 [1, 3, 5]
y = torch.tensor([1., 3., 5.])
# y_hat = F.one_hot(y.long(), 6).float()

y_hat = torch.tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0.]])

y_hat.shape, y.shape, y_hat.dtype, y.dtype

(torch.Size([3, 6]), torch.Size([3]), torch.float32, torch.float32)

In [26]:
y_hat

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0.]])

In [27]:
y

tensor([1., 3., 5.])

In [28]:
loss(y_hat, y.long())

tensor(1.9596)

#### 理解 lambda 表达式

In [29]:
say_your_name = lambda name: print(f'I am {name}')

In [30]:
say_your_name('123')

I am 123
